In [126]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sympy
import quaternion

In [127]:
# path = "../data/tst_cal_005.csv"
path = "data5_tst_cal_004_onArmArbitraryMotions.csv"
df = pd.read_csv(path)
df.head(0)

,IMU_TIMESTAMP(0)_0,IMU_GYROSCOPE(0)_0,IMU_GYROSCOPE(0)_1,IMU_GYROSCOPE(0)_2,IMU_GYROSCOPE(1)_0,IMU_GYROSCOPE(1)_1,IMU_GYROSCOPE(1)_2,IMU_ACCELEROMETER(0)_0,IMU_ACCELEROMETER(0)_1,IMU_ACCELEROMETER(0)_2,...,IMU_LINEAR_ACCELERATION(1)_1,IMU_LINEAR_ACCELERATION(1)_2,IMU_QUATERNION(0)_0,IMU_QUATERNION(0)_1,IMU_QUATERNION(0)_2,IMU_QUATERNION(0)_3,IMU_QUATERNION(1)_0,IMU_QUATERNION(1)_1,IMU_QUATERNION(1)_2,IMU_QUATERNION(1)_3


## Define equations

In [128]:
J       = sympy.MatrixSymbol('J', 4, 10)
phi     = sympy.MatrixSymbol('phi', 4, 1)
epsilon = sympy.MatrixSymbol('epsilon', 10, 1)

gauss_newton_delta = (J*J.transpose()) ** -1 * J * epsilon
gauss_newton_delta

(J*J.T)**(-1)*J*epsilon

In [129]:
theta1,rho1,theta2,rho2 = sympy.symbols('theta_1 rho_1 theta_2 rho_2')
j1 = sympy.Matrix([
        sympy.sin(theta1)*sympy.cos(rho1),
        sympy.sin(theta1)*sympy.sin(rho1),
        sympy.cos(theta1)
    ])
j2 = sympy.Matrix([
        sympy.sin(theta2)*sympy.cos(rho2),
        sympy.sin(theta2)*sympy.sin(rho2),
        sympy.cos(theta2)
    ])
omegaX,omegaY,omegaZ = sympy.symbols('omega_x omega_y omega_z')
omegaR = sympy.Matrix([omegaX, omegaY, omegaZ ])

error = omegaR.dot(j1.cross(j2))
derr_theta1 = sympy.diff(error,theta1)
derr_rho1   = sympy.diff(error,rho1)
derr_theta2 = sympy.diff(error,theta2)
derr_rho2   = sympy.diff(error,rho2)

J_col = sympy.Matrix(4,1, [
    derr_theta1, derr_rho1, derr_theta2, derr_rho2
])
derr_theta1

omega_x*(sin(rho_1)*cos(theta_1)*cos(theta_2) + sin(rho_2)*sin(theta_1)*sin(theta_2)) + omega_y*(-sin(theta_1)*sin(theta_2)*cos(rho_2) - cos(rho_1)*cos(theta_1)*cos(theta_2)) + omega_z*(-sin(rho_1)*sin(theta_2)*cos(rho_2)*cos(theta_1) + sin(rho_2)*sin(theta_2)*cos(rho_1)*cos(theta_1))

## Agorithm

In [130]:
th1_t = np.pi/2
rh1_t = np.pi/2
th2_t = np.pi/2
rh2_t = np.pi/2

rotV1 = (j1.subs([ (theta1,th1_t), (rho1,rh1_t) ])).evalf()
rotV2 = (j2.subs([ (theta2,th2_t), (rho2,rh2_t) ])).evalf()

RMS = 0
error_vector = []

Jacobian = None

for i,row in df.iterrows():
    ## Retrieve data from csv file
    omega1 = np.array([ row['IMU_GYROSCOPE(0)_0'], row['IMU_GYROSCOPE(0)_1'], row['IMU_GYROSCOPE(0)_2']] )
    omega2 = np.array([ row['IMU_GYROSCOPE(1)_0'], row['IMU_GYROSCOPE(1)_1'], row['IMU_GYROSCOPE(1)_2']] )
    q1  = np.quaternion(row['IMU_QUATERNION(0)_0'], row['IMU_QUATERNION(0)_1'], row['IMU_QUATERNION(0)_2'], row['IMU_QUATERNION(0)_3'])
    q2  = np.quaternion(row['IMU_QUATERNION(1)_0'], row['IMU_QUATERNION(1)_1'], row['IMU_QUATERNION(1)_2'], row['IMU_QUATERNION(1)_3'])
    ## Compute relative values
    qR = q1.inverse() * q2
    omegaR = quaternion.rotate_vectors(qR,omega2) - omega1
    ## Calculate error
    values = [
        (omegaX,omegaR[0]),(omegaY,omegaR[1]),(omegaZ,omegaR[2]),
        (theta1,th1_t),(rho1,rh1_t),(theta2,th2_t),(rho2,rh2_t)
    ]
    error_val = error.subs(values)
    error_vector.append(error_val)
    RMS += float(error_val*error_val)
    ## Calculate Jacobian
    if Jacobian is None:
        Jacobian = sympy.Matrix(J_col.subs(values))
    else:
        Jacobian = Jacobian.row_join(J_col.subs(values))
    




RMS = np.sqrt(RMS)
Jacobian

Matrix([
[ 5.43940552023626,  9.81090031275438,  6.77140535436711, -0.51132281978555, -5.18409277950933, -2.85552669032279,  0.626070153335922,  1.66683029138485,    3.806362562447,  3.31957317320437,  1.48656320440591,  2.16468787904972,  1.69588296651198, -0.405287208010072,  -0.0350556053323,  3.32620635031959,  5.82392246395087,  5.58800407351786,  2.10406609388778,  1.61838042100894,   5.29781540319685,  5.64337494575794,  1.01566272049601, -0.517159624980826,  2.03332590270651, -0.0202160773684233, -1.13898957641971,  2.53572454231806,  2.02813423713045,  -0.714724319431676,   -2.72061369206073, -2.01041535512157,  3.09807418229886,    4.66524505593409, -0.272829165541382,    -2.34060361047, -0.400719225735466,   2.70104753830441,  3.06941601478741,  1.27624044432086, -0.698442534998567, -4.36827657745765, -6.00779307754169, -1.24966998544648,  3.43998816187286,  2.16947031133116, -9.17586951538342, -25.0921630370059, -40.7968065924636,  -31.555868439863,  -13.629329611253,  -10.

## Compute C code

In [131]:
from sympy.utilities.codegen import codegen
# [(c_name, c_code), (h_name, c_header)] = codegen(('error', error), "C99", "error_fnc", header=False, empty=False)
# [(c_name, c_code), (h_name, c_header)] = codegen(('gauss_newton', gauss_newton_delta), "C99", "gauss_newton", header=False, empty=False)
[(c_name, c_code), (h_name, c_header)] = codegen(('error_dt1', derr_theta1), "C99", "error_derivatives", header=False, empty=False)
with open(c_name,'a+') as c_file:
    c_file.write(c_code)
with open(h_name,'a+') as h_file:
    h_file.write(c_header)

[(c_name, c_code), (h_name, c_header)] = codegen(('error_dr1', derr_rho1),   "C99", "error_derivatives", header=False, empty=False)
with open(c_name,'a+') as c_file:
    c_file.write(c_code)
with open(h_name,'a+') as h_file:
    h_file.write(c_header)

[(c_name, c_code), (h_name, c_header)] = codegen(('error_dt2', derr_theta2), "C99", "error_derivatives", header=False, empty=False)
with open(c_name,'a+') as c_file:
    c_file.write(c_code)
with open(h_name,'a+') as h_file:
    h_file.write(c_header)

[(c_name, c_code), (h_name, c_header)] = codegen(('error_dr2', derr_rho2),   "C99", "error_derivatives", header=False, empty=False)
with open(c_name,'a+') as c_file:
    c_file.write(c_code)
with open(h_name,'a+') as h_file:
    h_file.write(c_header)
